### Convolutional Neural Networks

###### Keywords: LRN, L2 vs L1, Data Augmentation, CIFAR-10

Advanced tricks will be used to build state-of-the-art CNN:<br>
* L2 normalization.
* Create more samples by flipping, and corpping of images. 
* Utilize LRN for each conv-pooling layer, which increased the generalization of the model.

Next I am going to implement this advanced CNN. At first, make sure the CIFAR10 related modules are cloned in your directory.

In [1]:
import cifar10
import cifar10_input
import numpy as np
import time
import tensorflow as tf

max_steps = 3000
batch_size = 128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'

def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var

tf.app.flags.DEFINE_string('f', '', 'kernel')
cifar10.maybe_download_and_extract()

In [2]:
image_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size) # Data Augmentation with 16 threading.
image_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [3]:
weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=0.05, wl=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1,1,1,1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/0.9, beta=0.75)

In [4]:
weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=0.05, wl=0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/0.9, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

In [5]:
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value

weight3 = variable_with_weight_loss(shape=[dim,384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bias3)

In [6]:
weight4 = variable_with_weight_loss(shape=[384,192], stddev=0.04, wl=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4)+bias4)

In [7]:
weight5 = variable_with_weight_loss(shape=[192,10], stddev=1/192, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4,weight5),bias5)

Conv-Pooling-FCN inference.

In [8]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

loss = loss(logits, label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

tf.train.start_queue_runners() # start 16 threading queues for data augmentation

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 6540)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 3828)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 6264)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 4812)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 6972)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 1916)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 3316)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daem

In [ ]:
for step in range(max_steps):
    start_time = time.time()
    image_batch, labels_batch = sess.run([image_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict={image_holder: image_batch, label_holder:labels_batch})
    duration = time.time() - start_time
    
    if step % 10 == 0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

In [ ]:
num_examples = 10000

import math

i = int(math.ceil(num_examples / batch_size))
true = 0
total = i * batch_size
step = 0
while step < i:
    image_batch, labels_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch, label_holder:label_batch})
    true += np.sum(predictions)
    step += 1
    
precision = true / total
print('Precision @1 = %.3f'% presision)

Data augmentation boost the sample size. Conventional machine learning algorithm will not be a good performance while the data is growing. So large neural networks can be data-driven.<br>

CNN serves as Feature Extration + Classification, but keep in mind I am on the way to avoid overfitting, converge accelerating, and improve generalization, etc.  